In [ ]:
%run main.py
%load_ext autoreload
%autoreload 2

!mkdir -p {NAVEC_DIR} {MODEL_DIR} {PACK_DIR}
s3 = S3()

In [ ]:
if not exists(NAVEC):
    !wget {NAVEC_URL} -O {NAVEC}
    s3.download(S3_TAGS_VOCAB, TAGS_VOCAB)
    s3.download(S3_MODEL_SHAPE, MODEL_SHAPE)
    s3.download(S3_MODEL_ENCODER, MODEL_ENCODER)
    s3.download(S3_MODEL_MORPH, MODEL_MORPH)

In [ ]:
navec = Navec.load(NAVEC)

words_vocab = Vocab(navec.vocab.words)
shapes_vocab = Vocab([PAD] + SHAPES)
tags_vocab = Vocab.load(TAGS_VOCAB)

word = NavecEmbedding(navec)
shape = Embedding(
    vocab_size=len(shapes_vocab),
    dim=SHAPE_DIM,
    pad_id=shapes_vocab.pad_id
)
emb = TagEmbedding(word, shape)
encoder = TagEncoder(
    input_dim=emb.dim,
    layer_dims=LAYER_DIMS,
    kernel_size=KERNEL_SIZE,
)
morph = MorphHead(encoder.dim, len(tags_vocab))
model = Morph(emb, encoder, morph)
model.eval()

model.emb.shape.load(MODEL_SHAPE)
model.encoder.load(MODEL_ENCODER)
model.head.load(MODEL_MORPH)

In [ ]:
model = model.to_exec()
model = model.strip_navec()
arrays, model = model.separate_arrays()

In [ ]:
with DumpPack(PACK) as pack:
    meta = Meta(ID)
    pack.dump_meta(meta)
    
    pack.dump_model(model)
    pack.dump_arrays(arrays)

    pack.dump_vocab(words_vocab, WORD)
    pack.dump_vocab(shapes_vocab, SHAPE)
    pack.dump_vocab(tags_vocab, TAG)

In [ ]:
morph = api.Morph.load(PACK)
morph.navec(navec)
words = 'Ежедневно очаги коронавирусной инфекции'.split()
markup = morph(words)
show_morph_markup(markup)

In [ ]:
s3.upload(PACK, S3_PACK)